In [ ]:
# Import necessary packages
from simulate_data import simulate_data
from build_smc import bi_level_SMC  
import pickle 
import numpy

In [ ]:
# Build and run the SMC for bi-level variable selection

# Parameters :
# - p_group: Number of group variables
# - p_ext: Number of external variables
# - p_ind: Number of individual variables
# - cov_var: Value of the variance-covariance matrix
# - n: Number of observations
# - nprocs: Number of cores used to launch the algorithm 
# - N: Number of particles
# - P: Length of the Markov chain
# - nruns: Number of runs
# - approximation_method: Likelihood approximation method ('ALA' or 'LA')
# - pi_ind: Parameter of the Bernoulli prior distribution for individual variables
# - pi_group: Parameter of the Bernoulli prior distribution for group variables

p_ind = 50
p_group = 5
p_ext = 5
cov_var = 0.5
n = 100

# Simulate data
data = simulate_data(p_ind=p_ind, 
                     p_group=p_group, 
                     p_ext=p_ext, 
                     cov_var=cov_var,
                     n=n)

# Build and run SMC
smc = bi_level_SMC(data=data,
                   p_group=p_group, 
                   p_ind=p_ind, 
                   nprocs=-5,
                   N=10000,
                   P=1000,
                   nruns=5,
                   approximation_method='ALA',
                   pi_ind=0.5,
                   pi_group=0.5)

In [ ]:
# Save the results
file = open('bi_level_SMC_results.pkl', 'wb')
pickle.dump(smc, file)
file.close()

In [ ]:
# Import the results
with open('bi_level_SMC_results.pkl', 'rb') as file:
    smc_results = pickle.load(file)

In [ ]:
# Get posterior probabilities of inclusion
run = 0
selection = numpy.mean(smc_results[run]['output'].X.theta, axis=0)
print('Posterior probability of inclusion:')
print(' ')
print('Groups:')
print(selection[:p_group])
print(' ')
print('Individual variables:')
print(selection[p_group:])